### 4.2 Saliency Mapping - Guided Grad-CAM

Guided gradient-weighted class activation mapping (guided Grad-CAM) was performed to generate activation maps (also known as saliency maps) highlighting the features that have a positive influence on the prediction of the class of interest. We adapted the code from [Utku Ozbulak](https://github.com/utkuozbulak/pytorch-cnn-visualizations) to implement guided Grad-CAM. Features that had postive impact on the CNN prediction for each target class were highlighted in bright white.

This notebook also includes the implementation of Vanilla backprop, Guided backprop and Grad-CAM.

In [1]:
import os

import numpy as np
import pandas as pd
from torchvision import transforms
import cv2
from PIL import Image

import torch
torch.manual_seed(123456789)
import torch.nn as nn
from torch.autograd import Variable

In [2]:
IMG_DIR = '/home/ziqi/Desktop/data/test_seg/blobs/'
MODEL_DIR = 'utils/CNN_model_parameters.pkl'
SAVE_DIR = '/home/ziqi/Desktop/figures/selected_test_blobs/'

img_class = ['cored','diffuse','CAA']
norm = np.load('utils/normalization.npy').item()
norm

{'mean': array([0.77906426, 0.74919518, 0.77529276]),
 'std': array([0.13986633, 0.15931302, 0.17665639])}

In [3]:
file = pd.read_csv('utils/selected_test_blobs.csv')
image_list = list(file['tilename'])

In [4]:
import matplotlib.pyplot as plt
def imshow(inp, size=3, title=None):
    """Imshow for Tensor."""
    try:
        inp = inp.numpy().transpose((1, 2, 0))
    except:
        inp = inp.transpose((1, 2, 0))
    
    inp = np.clip(inp, 0, 1)
    fig = plt.figure(figsize=(size,size))
    ax = fig.subplots()
    try:
        ax.imshow(inp)
    except:
        ax.imshow(inp[:,:,0], cmap='gray')
    if title is not None:
        ax.set_title(title)
    plt.pause(0.001)  
  

In [5]:
def save_class_activation_on_image(org_img, activation_map, file_name, save_dir, img_class):
    """
        Saves cam activation map and activation map on the original image
    Args:
        org_img (PIL img): Original image
        activation_map (numpy arr): activation map (grayscale) 0-255
        file_name (str): File name of the exported image
        save_dir (str): Dir for saving
        img_classes (str): target class - cored, diffuse or CAA
    """
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    # Grayscale activation map
    path_to_file = os.path.join(save_dir, file_name+'_Cam_Grayscale_{}.jpg'.format(img_class))
    cv2.imwrite(path_to_file, activation_map)
    # Heatmap of activation map
    activation_heatmap = cv2.applyColorMap(activation_map, cv2.COLORMAP_HSV)
    path_to_file = os.path.join(save_dir, file_name+'_Cam_Heatmap_{}.jpg'.format(img_class))
    cv2.imwrite(path_to_file, activation_heatmap)
    # Heatmap on picture
    img_with_heatmap = np.float32(activation_heatmap) + np.float32(org_img)
    img_with_heatmap = img_with_heatmap / np.max(img_with_heatmap)
    path_to_file = os.path.join(save_dir, file_name+'_Cam_On_Image_{}.jpg'.format(img_class))
    cv2.imwrite(path_to_file, np.uint8(255 * img_with_heatmap))
    
def get_positive_negative_saliency(gradient):
    """
        Generates positive and negative saliency maps based on the gradient
    Args:
        gradient (numpy arr): Gradient of the operation to visualize
    returns:
        pos_saliency ( )
    """
    pos_saliency = (np.maximum(0, gradient) / gradient.max())
    neg_saliency = (np.maximum(0, -gradient) / -gradient.min())
    return pos_saliency, neg_saliency

def convert_to_grayscale(cv2im):
    """
        Converts 3d image to grayscale
    Args:
        cv2im (numpy arr): RGB image with shape (D,W,H)
    returns:
        grayscale_im (numpy_arr): Grayscale image with shape (1,W,D)
    """
    grayscale_im = np.sum(np.abs(cv2im), axis=0)
    im_max = np.percentile(grayscale_im, 99)
    im_min = np.min(grayscale_im)
    grayscale_im = (np.clip((grayscale_im - im_min) / (im_max - im_min), 0, 1))
    grayscale_im = np.expand_dims(grayscale_im, axis=0)
    return grayscale_im

def save_gradient_images(gradient, file_name, save_dir, size=3, save=False, show=False):
    """
        Exports the original gradient image
    Args:
        gradient (np arr): Numpy array of the gradient with shape (3, 224, 224)
        file_name (str): File name to be exported
        save_dir (str): Dir for saving
        size (int): Size for showing
        save (bool): Whether save image
        show (bool): Whether show image
    """
    
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    gradient = gradient - gradient.min()
    gradient /= gradient.max()
    if show:
        imshow(gradient, size=size, title=file_name)
    gradient = np.uint8(gradient * 255).transpose(1, 2, 0)
    path_to_file = os.path.join(save_dir, file_name + '.jpg')
    # Convert RBG to GBR
    gradient = gradient[..., ::-1]
    if save:
        cv2.imwrite(path_to_file, gradient)


In [6]:
class CamExtractor():
    """
        Extracts cam features from the model
    """
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None

    def save_gradient(self, grad):
        self.gradients = grad

    def forward_pass_on_convolutions(self, x):
        """
            Does a forward pass on convolutions, hooks the function at given layer
        """
        conv_output = None
        for module_pos, module in self.model.features._modules.items():
            x = module(x)  # Forward
            if int(module_pos) == self.target_layer:
                x.register_hook(self.save_gradient)
                conv_output = x  # Save the convolution output on that layer
        return conv_output, x

    def forward_pass(self, x):
        """
            Does a full forward pass on the model
        """
        conv_output, x = self.forward_pass_on_convolutions(x)
        x = x.view(x.size(0), -1)
        x = self.model.classifier(x)
        return conv_output, x

class GradCam():
    """
        Produces class activation map
    """
    def __init__(self, model, target_layer):
        self.model = model
        self.model.eval()
        self.extractor = CamExtractor(self.model, target_layer)

    def generate_cam(self, input_image, target_class=None):
        # Full forward pass
        conv_output, model_output = self.extractor.forward_pass(input_image)
        if target_class is None:
            target_class = np.argmax(model_output.data.numpy())
        # Target for backprop
        one_hot_output = torch.FloatTensor(1, model_output.size()[-1]).zero_()
        one_hot_output[0][target_class] = 1
        # Zero grads
        self.model.features.zero_grad()
        self.model.classifier.zero_grad()
        # Backward pass with specified target
        model_output.backward(gradient=one_hot_output, retain_graph=True)
        # Get hooked gradients
        guided_gradients = self.extractor.gradients.data.numpy()[0]
        # Get convolution outputs
        target = conv_output.data.numpy()[0]
        # Get weights from gradients
        weights = np.mean(guided_gradients, axis=(1, 2))  # Take averages for each gradient
        # Create empty numpy array for cam
        cam = np.ones(target.shape[1:], dtype=np.float32)
        # Multiply each weight with its conv output and then, sum
        for i, w in enumerate(weights):
            cam += w * target[i, :, :]
        cam = cv2.resize(cam, (256, 256))
        cam = np.maximum(cam, 0)
        cam = (cam - np.min(cam)) / (np.max(cam) - np.min(cam))  # Normalize between 0-1
        cam = np.uint8(cam * 255)  # Scale between 0-255 to visualize
        return cam

In [7]:
from torch.nn import ReLU

class GuidedBackprop():
    """
       Produces gradients generated with guided back propagation from the given image
    """
    def __init__(self, model):
        self.model = model
        self.gradients = None
        self.model.eval()
        self.update_relus()
        self.hook_layers()

    def hook_layers(self):
        def hook_function(module, grad_in, grad_out):
            self.gradients = grad_in[0]

        # Register hook to the first layer
        first_layer = list(self.model.features._modules.items())[0][1]
        first_layer.register_backward_hook(hook_function)

    def update_relus(self):
        """
            Updates relu activation functions so that it only returns positive gradients
        """
        def relu_hook_function(module, grad_in, grad_out):
            """
            If there is a negative gradient, changes it to zero
            """
            if isinstance(module, ReLU):
                return (torch.clamp(grad_in[0], min=0.0),)
        # Loop through layers, hook up ReLUs with relu_hook_function
        for pos, module in self.model.features._modules.items():
            if isinstance(module, ReLU):
                module.register_backward_hook(relu_hook_function)

    def generate_gradients(self, input_image, target_class):
        # Forward pass
        model_output = self.model(input_image)
        # Zero gradients
        self.model.zero_grad()
        # Target for backprop
        one_hot_output = torch.FloatTensor(1, model_output.size()[-1]).zero_()
        one_hot_output[0][target_class] = 1
        # Backward pass
        model_output.backward(gradient=one_hot_output)
        # Convert Pytorch variable to numpy array
        gradients_as_arr = self.gradients.data.numpy()[0]
        return gradients_as_arr

In [8]:
class VanillaBackprop():
    """
        Produces gradients generated with vanilla back propagation from the image
    """
    def __init__(self, model):
        self.model = model
        self.gradients = None
        self.model.eval()
        # Hook the first layer to get the gradient
        self.hook_layers()

    def hook_layers(self):
        def hook_function(module, grad_in, grad_out):
            self.gradients = grad_in[0]

        # Register hook to the first layer
        first_layer = list(self.model.features._modules.items())[0][1]
        first_layer.register_backward_hook(hook_function)

    def generate_gradients(self, input_image, target_class):
        # Forward
        model_output = self.model(input_image)
        # Zero grads
        self.model.zero_grad()
        # Target for backprop
        one_hot_output = torch.FloatTensor(1, model_output.size()[-1]).zero_()
        one_hot_output[0][target_class] = 1
        # Backward pass
        model_output.backward(gradient=one_hot_output)
        # Convert Pytorch variable to numpy array
        gradients_as_arr = self.gradients.data.numpy()[0]
        return gradients_as_arr

In [9]:
def guided_grad_cam(grad_cam_mask, guided_backprop_mask):
    """
        Guided grad cam is just pointwise multiplication of cam mask and
        guided backprop mask
    Args:
        grad_cam_mask (np_arr): Class activation map mask
        guided_backprop_mask (np_arr):Guided backprop mask
    """
    cam_gb = np.multiply(grad_cam_mask, guided_backprop_mask)
    return cam_gb

In [10]:
class Net(nn.Module):

    def __init__(self, fc_nodes=512, num_classes=3, dropout=0.5):
        super(Net, self).__init__()

    def forward(self, x):
 
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)

        return x

# instatiate the model and load the model onto the CPU
model = torch.load(MODEL_DIR, map_location=lambda storage, loc: storage)

/home/ziqi/miniconda3/envs/pytorch/lib/python3.6/site-packages/torch/serialization.py:367: SourceChangeWarning: source code of class 'torch.nn.parallel.data_parallel.DataParallel' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/ziqi/miniconda3/envs/pytorch/lib/python3.6/site-packages/torch/serialization.py:367: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/ziqi/miniconda3/envs/pytorch/lib/python3.6/site-packages/torch/serialization.py:367: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can 

In [11]:
for img in image_list:
    
    if img is np.nan:
        continue
        
    wsi_name = img.split('/')[0]
    source_name = ''.join(img.split('/')[-1].split('.jpg'))
    img_name = wsi_name+'/'+source_name+'.jpg'
    save_dir = SAVE_DIR+'{}/'.format(source_name)
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    
    original_image = cv2.imread(IMG_DIR+img_name, 1)
    im = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
    im = Image.fromarray(im)    
    imtensor = transforms.ToTensor()(im)
    imtensor = transforms.Normalize(norm['mean'], norm['std'])(imtensor)
    imtensor = imtensor.view(1,3,256,256)
    input_img = Variable(imtensor, requires_grad=True)
    
    for target_class in range(len(img_class)):
        # Vanilla backprop
        VBP = VanillaBackprop(model.module.cpu())
        vanilla_grads = VBP.generate_gradients(input_img, target_class) # 0 for cored
        save_gradient_images(vanilla_grads, source_name + '_Vanilla_BP_color_'+img_class[target_class], 
                             save_dir, save=True)
        grayscale_vanilla_grads = convert_to_grayscale(vanilla_grads)
        save_gradient_images(grayscale_vanilla_grads, source_name + '_Vanilla_BP_gray_'+img_class[target_class], 
                             save_dir, save=True)

        # Guided backprop
        GBP = GuidedBackprop(model.module.cpu())
        guided_grads = GBP.generate_gradients(input_img, target_class)
        save_gradient_images(guided_grads, source_name + '_Guided_BP_color_'+img_class[target_class],
                             save_dir, save=True)
        grayscale_guided_grads = convert_to_grayscale(guided_grads)    # Convert to grayscale
        save_gradient_images(grayscale_guided_grads, source_name + '_Guided_BP_gray_'+img_class[target_class], 
                             save_dir, save=True)

        # Grad cam
        grad_cam = GradCam(model.module.cpu(), target_layer=23)
        cam = grad_cam.generate_cam(input_img, target_class)
        save_class_activation_on_image(original_image, cam, source_name, save_dir, img_class[target_class])

        # Guided Grad cam
        gcv2 = GradCam(model.module.cpu(), target_layer=23)
        cam = gcv2.generate_cam(input_img, target_class)

        GBP = GuidedBackprop(model.module.cpu())
        guided_grads = GBP.generate_gradients(input_img, target_class)

        cam_gb = guided_grad_cam(cam, guided_grads)
        save_gradient_images(cam_gb, source_name + '_GGrad_Cam_'+img_class[target_class], 
                             save_dir, save=True)
        grayscale_cam_gb = convert_to_grayscale(cam_gb)
        save_gradient_images(grayscale_cam_gb, source_name + '_GGrad_Cam_gray_'+img_class[target_class], 
                             save_dir, save=True)

print('done')

done
